<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入相关数据" data-toc-modified-id="导入相关数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入相关数据</a></span></li></ul></div>

# 导入相关的包

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn import manifold
from matplotlib import offsetbox
import matplotlib.pyplot as plt
%matplotlib inline

# 导入相关数据

In [2]:
trains = pd.read_csv("../clean_data/base_train.csv")
vals = pd.read_csv('../clean_data/model_fill_train.csv')
tests = pd.read_csv('../clean_data/testB.csv')
trains = pd.concat([trains, vals], axis=0, ignore_index=True)

In [3]:
features = [x for x in list(trains.columns) if x not in ["id","date",'blood_sugar','blood_sugar_log']]
x = np.array(trains[features])
y = np.array(trains["blood_sugar"])
n_samples, n_features = x.shape
n_neighbors = 30

In [4]:
trains.columns.shape

(37,)

In [5]:
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)     
    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(np.array(trains["blood_sugar"])[i]),
                 fontdict={'weight': 'bold', 'size': 9})
    if hasattr(offsetbox, 'AnnotationBbox'):
        ## only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(n_samples):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                ## don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [6]:
%%time
tsne = manifold.TSNE(n_components=3, init='pca', random_state=0)
# tsne.fit(X= x, y=y)
train_tsne = tsne.fit_transform(x)
test_tsne = tsne.fit_transform(np.array(tests[features]))

Wall time: 33min 13s


In [7]:
t = tsne.fit_transform(x, y)

KeyboardInterrupt: 

In [ ]:
t

In [8]:
train_tsne
test_tsne

array([[  76.5691  ,  -71.19027 ,  -22.309017],
       [ 113.2338  ,  -35.683113,  -14.120906],
       [-125.82636 ,  -44.984936,   19.15695 ],
       ...,
       [ 113.31938 ,  -33.82543 ,   37.320126],
       [-127.90981 ,   87.06086 ,  -13.236925],
       [ -81.648346,  -72.6307  ,   10.926115]], dtype=float32)

In [9]:
pd_train_tsne = pd.DataFrame(train_tsne,columns=["tsne_1","tsne_2","tsne_3"])
pd_train_tsne["id"] = trains["id"]
pd_test_tsne = pd.DataFrame(test_tsne,columns=["tsne_1","tsne_2","tsne_3"])
pd_test_tsne["id"] = tests["id"]

In [10]:
pd_train_tsne.head()
pd_train_tsne.to_csv("../raw_data/fea_train_2.csv",index=False)

In [11]:
pd_test_tsne.head()
pd_test_tsne.to_csv("../raw_data/fea_test_B_2.csv",index=False)